# Add batch effects

Say we are interested in identifying genes that differentiate between disease vs normal states.  However our dataset includes samples from different tissues or time points and there are variations in gene expression that are due to these other conditions and do not have to do with disease state.  These non-relevant variations in the data are called *batch effects*.  

We want to model these batch effects.  To do this we will:
1. Partition our simulated data into n batches
2. For each partition we will randomly shift the expression data.  We randomly generate a binary vector of length=number of genes (*offset vector*).  This vector will serve as the direction that we will shift to.  Then we also have a random scalar that will tell us how big of a step to take in our random direction (*stretch factor*).  We shift our partitioned data by: batch effect partition = partitioned data + stretch factor * offset vector
3. Repeat this for each partition
4. Append all batch effect partitions together


In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings(action='once')

from ggplot import *
from sklearn.decomposition import PCA
from numpy.random import seed
randomState = 123
seed(randomState)

/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Parameters
analysis_name = 'experiment_0'
NN_architecture = 'NN_2500_30'
num_PCs = 100
num_simulations = 10
num_batches = [1,2,5,10,20,50,100,500,1000,2000,3000,6000]
#num_batches = [1,2,3,4,5,6,7,8,9,10,15,20,50,100,500,800,1000,2000,3000,4000,5000,6000]

In [3]:
# Create directories
base_dir = os.path.abspath(os.path.join(os.getcwd(),"../.."))

new_dir = os.path.join(
    base_dir,
    "data",
    "batch_simulated")

analysis_dir = os.path.join(new_dir, analysis_name)

if os.path.exists(analysis_dir):
    print('directory already exists: {}'.format(analysis_dir))
else:
    print('creating new directory: {}'.format(analysis_dir))
os.makedirs(analysis_dir, exist_ok=True)

directory already exists: /home/alexandra/Documents/Repos/Batch_effects_simulation/data/batch_simulated/experiment_0


In [4]:
# Load arguments
simulated_data_file = os.path.join(
    base_dir,
    "data",
    "simulated",
    analysis_name,
    "simulated_data.txt")

umap_model_file = umap_model_file = os.path.join(
    base_dir,
    "models",  
    NN_architecture,
    "umap_model.pkl")

In [5]:
# Read in UMAP model
infile = open(umap_model_file, 'rb')
umap_model = pickle.load(infile)
infile.close()

In [6]:
# Read in data
simulated_data = pd.read_table(
    simulated_data_file,
    header=0, 
    index_col=0,
    sep='\t')

simulated_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
0,0.690384,0.642501,0.454786,0.650650,0.374857,0.414020,0.353704,0.566726,0.447497,0.165201,...,0.375559,0.600063,0.562897,0.640035,0.661894,0.325466,0.576836,0.567330,0.708088,0.615353
1,0.691060,0.655274,0.527149,0.680636,0.371842,0.443242,0.374220,0.533293,0.502785,0.165815,...,0.290522,0.591411,0.609883,0.585498,0.596831,0.162511,0.459705,0.531669,0.710235,0.698571
2,0.826005,0.694632,0.510346,0.642764,0.611429,0.517733,0.344205,0.631357,0.663116,0.199852,...,0.589395,0.581802,0.596350,0.671666,0.753091,0.160510,0.521740,0.511157,0.747738,0.728167
3,0.600721,0.564944,0.417176,0.594936,0.382868,0.440063,0.387032,0.466111,0.402363,0.223858,...,0.345520,0.547836,0.470917,0.460431,0.564647,0.256127,0.509963,0.348220,0.579351,0.583487
4,0.621544,0.615939,0.473489,0.599652,0.401605,0.481008,0.364476,0.444714,0.447605,0.215759,...,0.450535,0.532127,0.588728,0.547413,0.578212,0.234255,0.424777,0.501985,0.704726,0.717408
5,0.735929,0.732768,0.545550,0.632979,0.460602,0.437013,0.325476,0.572898,0.682289,0.203184,...,0.414369,0.576000,0.692773,0.643043,0.746655,0.215016,0.509911,0.638528,0.775373,0.746010
6,0.643262,0.647871,0.449706,0.552101,0.407846,0.421758,0.371682,0.551611,0.530684,0.185147,...,0.392800,0.503936,0.558234,0.505509,0.600357,0.213806,0.478426,0.443200,0.641380,0.686138
7,0.517200,0.567826,0.363922,0.483920,0.346046,0.440530,0.442479,0.474701,0.372429,0.156235,...,0.368144,0.488229,0.461496,0.446372,0.480050,0.364032,0.477898,0.388174,0.528046,0.492165
8,0.746649,0.745975,0.622174,0.748726,0.455695,0.499827,0.260720,0.625723,0.640932,0.206239,...,0.421102,0.636195,0.681262,0.676396,0.637278,0.241293,0.594934,0.663525,0.725401,0.708356
9,0.580710,0.586180,0.419463,0.592984,0.368153,0.405530,0.403313,0.445573,0.449738,0.212873,...,0.419540,0.534721,0.509361,0.491707,0.524512,0.257988,0.408664,0.429760,0.594038,0.572804


In [7]:
# Add batch effects
# ADD MULTIPLE SIMULATION RUNS
num_simulated_samples = simulated_data.shape[0]
num_genes = simulated_data.shape[1]
subset_genes_to_change = np.random.RandomState(randomState).choice([0, 1], size=(num_genes), p=[3./4, 1./4])
    
for i in num_batches:
    print('Creating simulated data with {} batches..'.format(i))
    
    batch_file = os.path.join(
            base_dir,
            "data",
            "batch_simulated",
            analysis_name,
            "Batch_"+str(i)+".txt")
    
    num_samples_per_batch = int(num_simulated_samples/i)
    
    if i == 1:        
        simulated_data.to_csv(batch_file, sep='\t')
        
    else:  
        batch_data_df = pd.DataFrame()
        
        simulated_data_draw = simulated_data
        for j in range(i):
            stretch_factor = np.random.uniform(1.0,1.5)
            
            # Randomly select samples
            batch_df = simulated_data_draw.sample(n=num_samples_per_batch, frac=None, replace=False)
            batch_df.columns = batch_df.columns.astype(str)
            
            # Update df to remove selected samples
            sampled_ids = list(batch_df.index)
            simulated_data_draw = simulated_data_draw.drop(sampled_ids)

            # Add batch effect
            subset_genes_to_change_tile = pd.DataFrame(
                pd.np.tile(
                    subset_genes_to_change,
                    (num_samples_per_batch, 1)),
                index=batch_df.index)

            offset_vector = pd.DataFrame(subset_genes_to_change_tile*stretch_factor)
            offset_vector.columns = offset_vector.columns.astype(str)
            batch_df = batch_df + offset_vector
            
            #batch_df = batch_df*stretch_factor

            # if any exceed 1 then set to 1 since gene expression is normalized
            batch_df[batch_df>=1.0] = 1.0


            # Append batched together
            batch_data_df = batch_data_df.append(batch_df)

            # Select a new direction (i.e. a new subset of genes to change)
            np.random.shuffle(subset_genes_to_change)
            
        # Save
        batch_data_df.to_csv(batch_file, sep='\t')

Creating simulated data with 1 batches..
Creating simulated data with 2 batches..
partition 0
size of simulated data is (6000, 5549)
partition 1
size of simulated data is (3000, 5549)
Creating simulated data with 5 batches..
partition 0
size of simulated data is (6000, 5549)
partition 1
size of simulated data is (4800, 5549)
partition 2
size of simulated data is (3600, 5549)
partition 3
size of simulated data is (2400, 5549)
partition 4
size of simulated data is (1200, 5549)
Creating simulated data with 10 batches..
partition 0
size of simulated data is (6000, 5549)
partition 1
size of simulated data is (5400, 5549)
partition 2
size of simulated data is (4800, 5549)
partition 3
size of simulated data is (4200, 5549)
partition 4
size of simulated data is (3600, 5549)
partition 5
size of simulated data is (3000, 5549)
partition 6
size of simulated data is (2400, 5549)
partition 7
size of simulated data is (1800, 5549)
partition 8
size of simulated data is (1200, 5549)
partition 9
size of

partition 67
size of simulated data is (1980, 5549)
partition 68
size of simulated data is (1920, 5549)
partition 69
size of simulated data is (1860, 5549)
partition 70
size of simulated data is (1800, 5549)
partition 71
size of simulated data is (1740, 5549)
partition 72
size of simulated data is (1680, 5549)
partition 73
size of simulated data is (1620, 5549)
partition 74
size of simulated data is (1560, 5549)
partition 75
size of simulated data is (1500, 5549)
partition 76
size of simulated data is (1440, 5549)
partition 77
size of simulated data is (1380, 5549)
partition 78
size of simulated data is (1320, 5549)
partition 79
size of simulated data is (1260, 5549)
partition 80
size of simulated data is (1200, 5549)
partition 81
size of simulated data is (1140, 5549)
partition 82
size of simulated data is (1080, 5549)
partition 83
size of simulated data is (1020, 5549)
partition 84
size of simulated data is (960, 5549)
partition 85
size of simulated data is (900, 5549)
partition 86
s

partition 126
size of simulated data is (4488, 5549)
partition 127
size of simulated data is (4476, 5549)
partition 128
size of simulated data is (4464, 5549)
partition 129
size of simulated data is (4452, 5549)
partition 130
size of simulated data is (4440, 5549)
partition 131
size of simulated data is (4428, 5549)
partition 132
size of simulated data is (4416, 5549)
partition 133
size of simulated data is (4404, 5549)
partition 134
size of simulated data is (4392, 5549)
partition 135
size of simulated data is (4380, 5549)
partition 136
size of simulated data is (4368, 5549)
partition 137
size of simulated data is (4356, 5549)
partition 138
size of simulated data is (4344, 5549)
partition 139
size of simulated data is (4332, 5549)
partition 140
size of simulated data is (4320, 5549)
partition 141
size of simulated data is (4308, 5549)
partition 142
size of simulated data is (4296, 5549)
partition 143
size of simulated data is (4284, 5549)
partition 144
size of simulated data is (4272,

partition 282
size of simulated data is (2616, 5549)
partition 283
size of simulated data is (2604, 5549)
partition 284
size of simulated data is (2592, 5549)
partition 285
size of simulated data is (2580, 5549)
partition 286
size of simulated data is (2568, 5549)
partition 287
size of simulated data is (2556, 5549)
partition 288
size of simulated data is (2544, 5549)
partition 289
size of simulated data is (2532, 5549)
partition 290
size of simulated data is (2520, 5549)
partition 291
size of simulated data is (2508, 5549)
partition 292
size of simulated data is (2496, 5549)
partition 293
size of simulated data is (2484, 5549)
partition 294
size of simulated data is (2472, 5549)
partition 295
size of simulated data is (2460, 5549)
partition 296
size of simulated data is (2448, 5549)
partition 297
size of simulated data is (2436, 5549)
partition 298
size of simulated data is (2424, 5549)
partition 299
size of simulated data is (2412, 5549)
partition 300
size of simulated data is (2400,

partition 437
size of simulated data is (756, 5549)
partition 438
size of simulated data is (744, 5549)
partition 439
size of simulated data is (732, 5549)
partition 440
size of simulated data is (720, 5549)
partition 441
size of simulated data is (708, 5549)
partition 442
size of simulated data is (696, 5549)
partition 443
size of simulated data is (684, 5549)
partition 444
size of simulated data is (672, 5549)
partition 445
size of simulated data is (660, 5549)
partition 446
size of simulated data is (648, 5549)
partition 447
size of simulated data is (636, 5549)
partition 448
size of simulated data is (624, 5549)
partition 449
size of simulated data is (612, 5549)
partition 450
size of simulated data is (600, 5549)
partition 451
size of simulated data is (588, 5549)
partition 452
size of simulated data is (576, 5549)
partition 453
size of simulated data is (564, 5549)
partition 454
size of simulated data is (552, 5549)
partition 455
size of simulated data is (540, 5549)
partition 45

...

partition 2356
size of simulated data is (1288, 5549)
partition 2357
size of simulated data is (1286, 5549)
partition 2358
size of simulated data is (1284, 5549)
partition 2359
size of simulated data is (1282, 5549)
partition 2360
size of simulated data is (1280, 5549)
partition 2361
size of simulated data is (1278, 5549)
partition 2362
size of simulated data is (1276, 5549)
partition 2363
size of simulated data is (1274, 5549)
partition 2364
size of simulated data is (1272, 5549)
partition 2365
size of simulated data is (1270, 5549)
partition 2366
size of simulated data is (1268, 5549)
partition 2367
size of simulated data is (1266, 5549)
partition 2368
size of simulated data is (1264, 5549)
partition 2369
size of simulated data is (1262, 5549)
partition 2370
size of simulated data is (1260, 5549)
partition 2371
size of simulated data is (1258, 5549)
partition 2372
size of simulated data is (1256, 5549)
partition 2373
size of simulated data is (1254, 5549)
partition 2374
size of simul

partition 2508
size of simulated data is (984, 5549)
partition 2509
size of simulated data is (982, 5549)
partition 2510
size of simulated data is (980, 5549)
partition 2511
size of simulated data is (978, 5549)
partition 2512
size of simulated data is (976, 5549)
partition 2513
size of simulated data is (974, 5549)
partition 2514
size of simulated data is (972, 5549)
partition 2515
size of simulated data is (970, 5549)
partition 2516
size of simulated data is (968, 5549)
partition 2517
size of simulated data is (966, 5549)
partition 2518
size of simulated data is (964, 5549)
partition 2519
size of simulated data is (962, 5549)
partition 2520
size of simulated data is (960, 5549)
partition 2521
size of simulated data is (958, 5549)
partition 2522
size of simulated data is (956, 5549)
partition 2523
size of simulated data is (954, 5549)
partition 2524
size of simulated data is (952, 5549)
partition 2525
size of simulated data is (950, 5549)
partition 2526
size of simulated data is (948,

partition 2664
size of simulated data is (672, 5549)
partition 2665
size of simulated data is (670, 5549)
partition 2666
size of simulated data is (668, 5549)
partition 2667
size of simulated data is (666, 5549)
partition 2668
size of simulated data is (664, 5549)
partition 2669
size of simulated data is (662, 5549)
partition 2670
size of simulated data is (660, 5549)
partition 2671
size of simulated data is (658, 5549)
partition 2672
size of simulated data is (656, 5549)
partition 2673
size of simulated data is (654, 5549)
partition 2674
size of simulated data is (652, 5549)
partition 2675
size of simulated data is (650, 5549)
partition 2676
size of simulated data is (648, 5549)
partition 2677
size of simulated data is (646, 5549)
partition 2678
size of simulated data is (644, 5549)
partition 2679
size of simulated data is (642, 5549)
partition 2680
size of simulated data is (640, 5549)
partition 2681
size of simulated data is (638, 5549)
partition 2682
size of simulated data is (636,

partition 2819
size of simulated data is (362, 5549)
partition 2820
size of simulated data is (360, 5549)
partition 2821
size of simulated data is (358, 5549)
partition 2822
size of simulated data is (356, 5549)
partition 2823
size of simulated data is (354, 5549)
partition 2824
size of simulated data is (352, 5549)
partition 2825
size of simulated data is (350, 5549)
partition 2826
size of simulated data is (348, 5549)
partition 2827
size of simulated data is (346, 5549)
partition 2828
size of simulated data is (344, 5549)
partition 2829
size of simulated data is (342, 5549)
partition 2830
size of simulated data is (340, 5549)
partition 2831
size of simulated data is (338, 5549)
partition 2832
size of simulated data is (336, 5549)
partition 2833
size of simulated data is (334, 5549)
partition 2834
size of simulated data is (332, 5549)
partition 2835
size of simulated data is (330, 5549)
partition 2836
size of simulated data is (328, 5549)
partition 2837
size of simulated data is (326,

KeyboardInterrupt: 

## Plot batch data using UMAP

In [ ]:
"""
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # UMAP embedding of decoded batch data
    batch_data_UMAPencoded = umap_model.transform(batch_data)
    batch_data_UMAPencoded_df = pd.DataFrame(data=batch_data_UMAPencoded,
                                             index=batch_data.index,
                                             columns=['1','2'])
    
        
    g = ggplot(aes(x='1',y='2'), data=batch_data_UMAPencoded_df) + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    
    print(g)"""

## Plot batch data using PCA

In [ ]:
"""
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # PCA projection    
    pca = PCA(n_components=num_PCs)
    batch_data_PCAencoded = pca.fit_transform(batch_data)
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded,
                                         index=batch_data.index
                                         )
    
    g = sns.pairplot(batch_data_PCAencoded_df)
    g.fig.suptitle("Batch {}".format(i))
       
    # Select pairwise PC's to plot
    pc1 = 0
    pc2 = 2
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded[:,[pc1,pc2]],
                                         index=batch_data.index,
                                         columns=['PC {}'.format(pc1), 'PC {}'.format(pc2)])
    
    g = ggplot(aes(x='PC {}'.format(pc1),y='PC {}'.format(pc2)), data=batch_data_PCAencoded_df)  + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    print(g)

"""